# Flask Mini Project

In [1]:
!pip install -q flask pyngrok matplotlib

In [ ]:
import getpass
from pyngrok import ngrok, conf

print("Enter your ngrok authtoken (https://dashboard.ngrok.com/auth):")
conf.get_default().auth_token = getpass.getpass()


Enter your ngrok authtoken (https://dashboard.ngrok.com/auth):


In [ ]:
import os, textwrap

os.makedirs("/content/templates", exist_ok=True)
os.makedirs("/content/static", exist_ok=True)

# /content/static/styles.css
css = """
.chart-container {
    width: 80%;
    margin: 20px auto;
}
.error {
    color: #b00020;
    font-weight: 600;
}
"""
with open("/content/static/styles.css", "w") as f:
    f.write(textwrap.dedent(css).strip() + "\n")

# /content/templates/index.html
html = """
<!DOCTYPE html>
<html>
<head>
    <meta charset="utf-8">
    <title>Data Visualization with Flask</title>
    <link rel="stylesheet" href="{{ url_for('static', filename='styles.css') }}">
</head>
<body>
    <h1>Data Visualization with Flask</h1>

    {% if error %}
      <p class="error">{{ error }}</p>
    {% endif %}

    <form method="POST" action="/">
        <div>
            <label for="categories">Categories (comma-separated):</label>
            <input type="text" name="categories" required placeholder="A, B, C">
        </div>
        <div>
            <label for="values">Values (comma-separated):</label>
            <input type="text" name="values" required placeholder="10, 20, 30">
        </div>
        <button type="submit">Generate Bar Chart</button>
    </form>

    <div class="chart-container">
        {% if chart_url %}
            <img src="data:image/png;base64,{{ chart_url }}" alt="Bar Chart">
        {% endif %}
    </div>
</body>
</html>
"""
with open("/content/templates/index.html", "w") as f:
    f.write(textwrap.dedent(html).strip() + "\n")

print("Created:")
print("/content/static/styles.css")
print("/content/templates/index.html")


In [ ]:
import os
import io
import base64
import threading

import matplotlib
matplotlib.use("Agg")  # non-interactive backend for servers
import matplotlib.pyplot as plt

from flask import Flask, render_template, request
from pyngrok import ngrok

os.environ["FLASK_DEBUG"] = "development"

app = Flask(__name__, template_folder="/content/templates", static_folder="/content/static")
port = 5000

# Open ngrok tunnel
public_url = ngrok.connect(port).public_url
print(f" * ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:{port}\"")
app.config["BASE_URL"] = public_url

def generate_bar_chart(categories, values):
    fig, ax = plt.subplots(figsize=(8, 4))
    ax.bar(categories, values)
    ax.set_xlabel("Category")
    ax.set_ylabel("Value")
    ax.set_title("Bar Chart")
    fig.tight_layout()

    buf = io.BytesIO()
    plt.savefig(buf, format="png", dpi=150, bbox_inches="tight")
    plt.close(fig)
    buf.seek(0)
    return base64.b64encode(buf.read()).decode("utf-8")

@app.route("/", methods=["GET", "POST"])
def index():
    chart_url = None
    error = None

    if request.method == "POST":
        cats_raw = request.form.get("categories", "")
        vals_raw = request.form.get("values", "")

        categories = [c.strip() for c in cats_raw.split(",") if c.strip() != ""]
        try:
            values = [float(v.strip()) for v in vals_raw.split(",") if v.strip() != ""]
        except ValueError:
            error = "Values must be numeric (comma-separated)."
            return render_template("index.html", chart_url=None, error=error)

        if len(categories) == 0 or len(values) == 0:
            error = "Provide at least one category and one value."
        elif len(categories) != len(values):
            error = "Categories and values must have the same length."
        else:
            chart_url = generate_bar_chart(categories, values)

    return render_template("index.html", chart_url=chart_url, error=error)

# Start Flask server in a thread
threading.Thread(target=app.run, kwargs={"use_reloader": False, "host": "0.0.0.0", "port": port}, daemon=True).start()
print("App running. Open:", public_url)
